I have previously appplied several regression models and analyzed their results. They were not reliable enough to be used for real-world portfolio management.

Another way to approach the main problem of stock market investing is to consider it as a classification problem: can a model predict well enough if a stock price will increase by 10% or more in the next 7 days ?

Let's investigate by building the appropriate dataset, and train various classifiers.

In [5]:
!pip install -r requirements.txt

In [95]:
import utils
import pandas as pd
import numpy as np

In [22]:
X_df = utils.get_stock_feature_dataset('ALNOV.PA')

C:\ProgramData\Miniconda3\lib\site-packages\ta\trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\ProgramData\Miniconda3\lib\site-packages\ta\trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


In [48]:
X_df.head(8)

,Open,High,Low,Close,Adj Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,others_dlr,others_cr,cac40_Open,cac40_High,cac40_Low,cac40_Close,sbf120_Open,sbf120_High,sbf120_Low,sbf120_Close
Date,,,,,,,,,,,,,,,,,,,,,
2012-10-12,8.70,8.70,8.70,8.70,8.70,450,0.0,450,0.0,-24.874197,...,-2.561258,0.000000,3401.260010,3419.000000,3389.080078,3389.080078,2607.149902,2607.149902,2607.149902,2607.149902
2012-10-15,8.48,8.48,8.48,8.48,8.48,730,0.0,-280,0.0,-24.874197,...,-2.561258,-2.528736,3387.760010,3439.520020,3387.760010,3420.280029,2630.100098,2630.100098,2630.100098,2630.100098
2012-10-16,8.70,8.70,8.70,8.70,8.70,3000,0.0,2720,0.0,-24.874197,...,2.561258,0.000000,3443.600098,3500.939941,3428.969971,3500.939941,2686.030029,2686.030029,2686.030029,2686.030029
2012-10-17,8.72,8.72,8.72,8.72,8.72,800,0.0,3520,0.0,-24.874197,...,0.229621,0.229885,3503.820068,3530.790039,3494.350098,3527.500000,2704.060059,2704.060059,2704.060059,2704.060059
2012-10-18,8.83,8.83,8.83,8.83,8.83,138,0.0,3658,0.0,-24.874197,...,1.253578,1.494253,3527.760010,3542.580078,3508.399902,3535.179932,2709.370117,2709.370117,2709.370117,2709.370117
2012-10-19,8.85,8.85,8.85,8.85,8.85,1255,0.0,4913,0.0,-24.874197,...,0.226244,1.724138,3527.790039,3537.919922,3496.760010,3504.560059,2688.659912,2688.659912,2688.659912,2688.659912
2012-10-22,8.70,8.70,8.70,8.70,8.70,655,0.0,4258,0.0,-24.874197,...,-1.709443,0.000000,3498.379883,3519.050049,3475.820068,3483.250000,2672.540039,2672.540039,2672.540039,2672.540039
2012-10-23,8.48,8.48,8.48,8.48,8.48,200,0.0,4058,0.0,-24.874197,...,-2.561258,-2.528736,3485.939941,3492.020020,3391.300049,3406.500000,2616.760010,2616.760010,2616.760010,2616.760010


In [65]:
days_with_increase = 7
increase = 0.1 # float between 0 and 1, equivalent to the desired % increase when multiplied by 100
label_name = '{}_days_increase'.format(days_with_increase)

# Build the target dataset: label 1 if stock price increased by 10% or more in the following days, 0 otherwise
y_df = pd.DataFrame(index=X_df.index, columns=[label_name])
for i in range(len(X_df)):
    increase_threshold = X_df['Adj Close'].iloc[i] + increase * X_df['Adj Close'].iloc[i]
    y_df.iloc[i] = 0 if X_df['Adj Close'].iloc[i+1:i+1+days_with_increase].ge(increase_threshold).sum() == 0 else 1

In [87]:
y_df.tail()

,7_days_increase
Date,
2020-06-29,0
2020-06-30,1
2020-07-01,0
2020-07-02,1
2020-07-03,0


In [106]:
# Check that labels are correct:

# There was an important increase of our stock price on the market day just after April 9th 2020
print("Let's check if this {:.2f}% increase is correctly labeled...".format((X_df.loc['2020-04-14']['Adj Close'] - X_df.loc['2020-04-09']['Adj Close']) / X_df.loc['2020-04-09']['Adj Close'] * 100))

# Get index of April 9th 2020, the day before increase
idx = len(X_df.loc[:'2020-04-09']) - 1
assert np.array_equal(X_df.loc['2020-04-09'].values, X_df.iloc[idx].values)

# Check that its corresponding label is 1
assert y_df.iloc[idx][label_name] == 1

print('Good!')

Let's check if this 31.62% increase is correctly labeled...
Good!
